In [1]:
import pandas as pd
from prophet import Prophet
from pymongo import MongoClient
from prophet.plot import plot_plotly, plot_components_plotly
import os
import time
import pickle
from datetime import datetime
import itertools
import dask
from dask.distributed import Client
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

Importing plotly failed. Interactive plots will not work.


In [12]:
df_clusters = pd.read_csv('./clusters-covid.csv')
parents = list(df_clusters[df_clusters['is_parent'] == 1]['GISJOIN'])
print(parents)
# pickle.dump(parents, open('ucc-21/covid-parents.pkl', 'wb'))
len(set(parents))

['G0100790', 'G0400130', 'G0500330', 'G0500890', 'G0600010', 'G0600370', 'G0600590', 'G0600650', 'G0600730', 'G0800410', 'G0800690', 'G1200570', 'G1200710', 'G1200860', 'G1200990', 'G1300210', 'G1301170', 'G1600750', 'G1600810', 'G1700270', 'G1800950', 'G1801030', 'G1901670', 'G2100670', 'G2101110', 'G2700270', 'G3000310', 'G3400070', 'G3400290', 'G3600290', 'G3600590', 'G3601190', 'G3701190', 'G3900350', 'G3900490', 'G3901510', 'G4001090', 'G4200710', 'G4200910', 'G4201010', 'G4500310', 'G4500450', 'G4500790', 'G4600910', 'G4701710', 'G4800270', 'G4800290', 'G4801570', 'G4802010', 'G4803750', 'G4804390', 'G4900350', 'G4900490', 'G5300050', 'G5300350', 'G5500090']


56

In [20]:
# Load parent_child_map.pkl (Sanity Check)
parent_child_map = pickle.load(open('pickles/parent_child_map.pkl', 'rb'))
keys = parent_child_map.keys()
assert len(set(parents) - set(keys)) == 0

In [30]:
for p in parents:
    print(p, end=', ')

G0400130, G0400190, G0400270, G0500590, G0500690, G0501030, G0600370, G0600590, G0600650, G0600710, G0600730, G0800050, G0800150, G0900090, G1200090, G1200110, G1200170, G1200830, G1201050, G1301350, G1700310, G1700430, G1701110, G1800970, G1900610, G1901010, G2405100, G2601210, G2700810, G2900950, G3000130, G3200030, G3300150, G3400030, G3500010, G3600510, G3600590, G3600910, G3900170, G3900350, G3900410, G4001090, G4200030, G4201010, G4600990, G4800270, G4800290, G4800850, G4801090, G4801130, G4801570, G4802010, G4802150, G4900350, G5100090, G5400550, 

In [21]:
df_map = pickle.load(open('../pickles/covid/df_map.pkl', 'rb'))
df_cases = df_map['cases']
df_cases

,ds,y
0,2021-01-07,1757
1,2020-06-08,110
2,2020-10-25,265
3,2020-07-20,659
4,2020-12-27,6043
...,...,...
367,2020-03-14,2
368,2020-02-26,0
369,2020-03-06,0
370,2020-02-08,0


In [22]:
def predict(df_train):
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    # model.fit(df, algorithm='LBFGS')
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300)
    df_train_forecast = m.predict(df_train_future)

    return fit_m, df_train_future, df_train_forecast

# fit_m, df_train, df_train_forecast = predict(df_cases)

## Exhaustively train parents

In [23]:
db = MongoClient("lattice-100", 27018)
collection = 'covid_county_formatted'

def get_df_by_gis_join(gis_join):
    print(gis_join, end=' ')
    cursor = db.sustaindb[collection].aggregate([{"$match": {"GISJOIN": gis_join}}])
    df = pd.DataFrame(list(cursor))[['date', 'cases']]
    df.columns = ['ds','y']
    return df

def predict_parallel(df_train, gis_join):
    time1 = time.monotonic()
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)
    time2 = time.monotonic()

    return gis_join, (time2 - time1)


gis_joins_list = []
child_dfs_list = []
# cutoff = 3000

for parent in parents:
    gis_joins_list.append(parent)
    child_dfs_list.append(get_df_by_gis_join(parent))


def train_one_parent(df_, gis_join):
    gis_join, time_taken = predict_parallel(df_, gis_join)
    return gis_join, time_taken


# assert len(gis_joins_list) == len(parents)
print(f'#gis_joins: {len(gis_joins_list)}')
print(f'#child_dfs: {len(child_dfs_list)}')
assert len(gis_joins_list) == len(child_dfs_list)

G0100790 G0400130 G0500330 G0500890 G0600010 G0600370 G0600590 G0600650 G0600730 G0800410 G0800690 G1200570 G1200710 G1200860 G1200990 G1300210 G1301170 G1600750 G1600810 G1700270 G1800950 G1801030 G1901670 G2100670 G2101110 G2700270 G3000310 G3400070 G3400290 G3600290 G3600590 G3601190 G3701190 G3900350 G3900490 G3901510 G4001090 G4200710 G4200910 G4201010 G4500310 G4500450 G4500790 G4600910 G4701710 G4800270 G4800290 G4801570 G4802010 G4803750 G4804390 G4900350 G4900490 G5300050 G5300350 G5500090 #gis_joins: 56
#child_dfs: 56


In [24]:
client = Client('lattice-150:8786')

counter = 1
lazy_results = []
for gis_join, df_ in zip(gis_joins_list, child_dfs_list):
    lazy_result = dask.delayed(train_one_parent)(df_, gis_join)
    lazy_results.append(lazy_result)
    print(counter, end=', ')
    counter += 1

futures = dask.persist(*lazy_results)  # trigger computation in the background
results = dask.compute(*futures)
results[:5]

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 

(('G0100790', 11.903916573617607),
 ('G0400130', 8.80438134027645),
 ('G0500330', 8.751174662262201),
 ('G0500890', 8.48774425406009),
 ('G0600010', 8.59309938782826))

## GridSearch on Parent Models

In [25]:
def grid_search(df):
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    time1 = time.monotonic()
    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial= '100 days', period='100 days', horizon='100', parallel="dask")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses

    print(f'Time: {time.monotonic() - time1}')
    
    optimal_params = tuning_results[tuning_results['rmse'] == min(tuning_results['rmse'])]
    rmse = optimal_params['rmse'].iloc[0]
    changepoint_prior_scale = optimal_params['changepoint_prior_scale'].iloc[0]
    seasonality_prior_scale = optimal_params['seasonality_prior_scale'].iloc[0]
    
    return rmse, changepoint_prior_scale, seasonality_prior_scale


rmses = []
changepoint_prior_scales = []
seasonality_prior_scales = []

counter = 1
for parent in parents:
    df_parent = get_df_by_gis_join(parent)
    rmse, changepoint_prior_scale, seasonality_prior_scale = grid_search(df_parent)
    rmses.append(rmse)
    changepoint_prior_scales.append(changepoint_prior_scale)
    seasonality_prior_scales.append(seasonality_prior_scale)
    print(counter, end=', ')
    counter += 1

G0100790 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 3 forecasts with cutoffs between 2020-08-18 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


Initial log joint probability = -8.65829
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1      -8.65829             0       348.049       1e-12       0.001       24   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -156.249
Iteration  1. Log joint probability =    266.976. Improved by 423.225.
Iteration  2. Log joint probability =    446.114. Improved by 179.137.
Iteration  3. Log joint probability =    483.761. Improved by 37.6474.
Iteration  4. Log joint probability =    489.844. Improved by 6.08341.
Iteration  5. Log joint probability =    491.523. Improved by 1.67854.
Iteration  6. Log joint probability =    491.533. Improved by 0.00981339.
Iteration  7. Log joint probability =    491.815. Improved by 0.282026.
Iteration  8. Log joint probability =    491.832. Improved by 0.0170522.
Iteration  9. Log joint probability =    49

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -8.65829
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1      -8.65829             0       348.049       1e-12       0.001       24   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -170.064
Iteration  1. Log joint probability =    282.366. Improved by 452.431.
Iteration  2. Log joint probability =    404.533. Improved by 122.167.
Iteration  3. Log joint probability =    422.812. Improved by 18.2785.
Iteration  4. Log joint probability =    460.142. Improved by 37.3304.
Iteration  5. Log joint probability =    489.753. Improved by 29.6111.
Iteration  6. Log joint probability =    490.139. Improved by 0.385444.
Iteration  7. Log joint probability =    491.554. Improved by 1.41589.
Iteration  8. Log joint probability =    494.379. Improved by 2.8249.
Iteration  9. Log joint probability =    495.342.

...

Initial log joint probability = -10.9004
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       601.725     0.0521828       60.4047           1           1      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       603.144    0.00152186       11.0546           1           1      255   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     209        603.27   0.000344382       16.1146   2.092e-05       0.001      305  LS failed, Hessian reset 
     299        603.44    0.00292178       12.5569           1           1      430   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     315       603.457   0.000464059       7.93575   6.387e-05       0.001      497  LS failed, Hessian reset 
     366       603.509   3.66467e-05       5.61714     7.6e-06       0.001      595  LS failed, Hessian rese

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 3 forecasts with cutoffs between 2020-08-18 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


Initial log joint probability = -10.9004
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        601.68     0.0630095       18.9673           1           1      126   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     130       602.266   0.000364296       14.9987    3.37e-05       0.001      202  LS failed, Hessian reset 
     162       602.582   0.000687055       10.7348   0.0001082       0.001      278  LS failed, Hessian reset 
     199        603.08    0.00496866       30.0789      0.1106           1      331   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     261       603.379   0.000226369       10.2582   2.474e-05       0.001      444  LS failed, Hessian reset 
     295       603.424   0.000120599       9.33722   1.808e-05       0.001      522  LS failed, Hessian reset 
     299       603.426   0.000399291       5.75296      0.2033   

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>


Time: 209.96628956799395
56, 

In [26]:
print(len(rmses))
print(len(changepoint_prior_scales))
print(len(seasonality_prior_scales))

56
56
56


In [27]:
pickle.dump(rmses, open('pickles/rmses.pkl', 'wb'))
pickle.dump(changepoint_prior_scales, open('pickles/changepoint_prior_scales.pkl', 'wb'))
pickle.dump(seasonality_prior_scales, open('pickles/seasonality_prior_scales.pkl', 'wb'))

In [28]:
df_parent_weights = pd.DataFrame(zip(parents, rmses, changepoint_prior_scales, seasonality_prior_scales),
                                columns=["GISJOIN", "rmse", "changepoint_prior_scale", "seasonality_prior_scale"])
df_parent_weights.to_csv('covid_parents_trained.csv', index=False)